In [2]:
%matplotlib widget

from matplotlib import pyplot as plt
import os
from themachinethatgoesping.echosounders.simrad import FileRaw,FileRaw_mapped,SimradDatagram_type_from_string,datagram_type_to_string,t_SimradDatagramType
import themachinethatgoesping.echosounders.simrad as simrad
import themachinethatgoesping.echosounders.simrad.datagrams as dg
import themachinethatgoesping.tools.progressbars as prg
import themachinethatgoesping.tools as tools
import themachinethatgoesping.navigation.nmea_0183 as NMEA
import numpy
import os, sys
from time import time
from tqdm.auto import tqdm
from collections import defaultdict
import numpy as np

#load files
folders =  ["/home/data/"]
#folders =  ["/home/data/turbeams/"]
folders = ["/home/data/no_backup/ek80_test/"]

files = []
for folder in folders:
    for r,d,f in os.walk(folder):
        for file in f:
            if file.endswith('.raw'):
                files.append(r + '/' +file)
            
files.sort()
#files
file_name = files[0]

In [3]:
fm = FileRaw_mapped(files, show_progress=True)
fm

indexing files ⠐ 100% :00s<00m:00s] [Found: 12382 packages in 26 files (2617MB)]                                            


FileSimradRaw
#############
- Number of loaded .raw files: : 26      
- Total file size: :             2.56 GB 

 Detected datagrams 
-------------------- 
- Total:           12382 
- Packages [NME0]: 6511  [NMEA text datagram]
- Packages [XML0]: 2338  [XML0 text datagram]
- Packages [MRU0]: 1143  [Motion binary datagram]
- Packages [FIL1]: 104   [Filter binary datagram]
- Packages [RAW3]: 2286  [Sample binary datagram]

In [4]:
%%timeit -n1 -r1
A = {}
for nm in tqdm(fm("NME0",0,-1)):
    a=nm.decode()
    
    
    
    if nm.type == 'GGA' or nm.type == 'GLL' or nm.type == "RMC":
        a.get_latitude()
        a.get_longitude()
    
    continue
    
    if nm.type == 'ZDA':
        a = a.to_timestamp()
        pass
    
    if nm.type == 'RMC':
        pass
        
    if nm.type == 'GGA':
        pass
    if nm.type == 'GLL':
        pass
    if nm.type == 'HDT':
        pass
    if nm.type == 'GST':
        pass
    
    
        
    

  0%|          | 0/6511 [00:00<?, ?it/s]

31.2 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [5]:
t1 = time()
simrad.test_speed_raw(fm,"NME0")
#print(len(fm)/(time()-t1),'it/s')
t2 = time()
simrad.test_speed_type(fm,"NME0")
#print(len(fm)/(time()-t2),'it/s')
t3 = time()
simrad.test_speed_raw(fm,"NME0")
#print(len(fm)/(time()-t3),'it/s')
t4 = time()
simrad.test_speed_decode_nmea(fm)
#print(len(fm)/(time()-t2),'it/s')
t5 = time()
print(round(100*(t4-t3)/(t5-t4),2))
print('raw :',len(fm('NME0'))/(t4-t3),'it/s')
print('read:',len(fm('NME0'))/(t3-t2),'it/s')
print('pack:',len(fm('NME0'))/(t5-t4),'it/s')

test reading ⠠ 100% :00s<00m:00s] [time: 6.340878ms]                  
test reading ⠠ 100% :00s<00m:00s] [time: 5.608611ms]                  
test reading ⠠ 100% :00s<00m:00s] [time: 5.393595ms]                  
test reading ⠠ 100% :00s<00m:00s] [time: 7.918606ms]                  
70.94
raw : 1029095.7283792441 it/s
read: 1001801.6633895818 it/s
pack: 729995.0105319433 it/s


In [6]:
from collections import defaultdict
packages = defaultdict(list)
for nmea in tqdm(fm.i_NME0):
    t = nmea.name
    if t == "invalid":
        print(len(packages['invalid']),end='\r')
    else:
        t = nmea.sender + ' ' + nmea.type
        
    packages[t].append(nmea)
        
for k,v in packages.items():
    print(k,':',len(v))

  0%|          | 0/6511 [00:00<?, ?it/s]

SD VLW : 26
IN ZDA : 1622
IN GGA : 1621
IN VTG : 1621
IN HDT : 1621


In [7]:
for k,v in packages.items():
    print(k,':',len(v))

SD VLW : 26
IN ZDA : 1622
IN GGA : 1621
IN VTG : 1621
IN HDT : 1621
